# Lowering `graphblas.matrix_multiply` to Generic Form

This example will go over how to use the `--graphblas-structuralize` pass from `graphblas-opt` to lower `graphblas.matrix_multiply` ops to `graphblas.matrix_multiply_generic` ops.

`graphblas.matrix_multiply` ops specify the behavior of a matrix multiplication via semiring attributes. The `--graphblas-structuralize` pass will lower those ops into equivalent `graphblas.matrix_multiply_generic` ops with blocks specifying the behavior indicated by those semirings.

Let's first import some necessary libraries.

In [5]:
import tempfile
from mlir_graphblas.cli import GRAPHBLAS_OPT_EXE

Since [sparse tensor encodings](https://mlir.llvm.org/docs/Dialects/SparseTensorOps/#sparsetensorencodingattr) can be very verbose in MLIR, let's import some helpers to make the MLIR code more readable.

In [6]:
from mlir_graphblas.tools import tersify_mlir

## Example

Here's some example `graphblas.matrix_multiply` code using the "min_plus" semiring. 

In [7]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @mat_mul(%argA: tensor<?x?xf64, #CSR64>, %argB: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSR64> {
    %answer = graphblas.matrix_multiply %argA, %argB { semiring = "min_plus" } : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to tensor<?x?xf64, #CSR64>
    return %answer : tensor<?x?xf64, #CSR64>
}
"""

Let's see what code we get when we run it through `graphblas-opt` with the `--graphblas-structuralize` pass.

In [8]:
with tempfile.NamedTemporaryFile() as temp:
    temp_file_name = temp.name
    with open(temp_file_name, 'w') as f:
        f.write(mlir_text)
    temp.flush()

    output_mlir = ! cat $temp_file_name | $GRAPHBLAS_OPT_EXE --graphblas-structuralize
    output_mlir = "\n".join(output_mlir)
    output_mlir = tersify_mlir(output_mlir)

print(output_mlir)

#CSC64 = #sparse_tensor.encoding<{ 
    dimLevelType = [ "dense", "compressed" ], 
    dimOrdering = affine_map<(d0, d1) -> (d1, d0)>, 
    pointerBitWidth = 64, 
    indexBitWidth = 64 
}>

#CSR64 = #sparse_tensor.encoding<{ 
    dimLevelType = [ "dense", "compressed" ], 
    dimOrdering = affine_map<(d0, d1) -> (d0, d1)>, 
    pointerBitWidth = 64, 
    indexBitWidth = 64 
}>

builtin.module  {
  builtin.func @mat_mul(%arg0: tensor<?x?xf64, #CSR64>, %arg1: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSR64> {
    %cst = constant 1.7976931348623157E+308 : f64
    %0 = graphblas.matrix_multiply_generic %arg0, %arg1 {mask_complement = false} : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to tensor<?x?xf64, #CSR64>  {
      graphblas.yield add_identity %cst : f64
    },  {
    ^bb0(%arg2: f64, %arg3: f64):  // no predecessors
      %1 = cmpf olt, %arg2, %arg3 : f64
      %2 = select %1, %arg2, %arg3 : f64
      graphblas.yield add %2 : f64
    },  {
    ^bb0(%arg2: f64, %arg3: f64)

As shown above, `--graphblas-structuralize` expanded the "min_plus" semiring into blocks performing that exact behavior.

We'll leave exploring how the other semirings expand as an exercise for the reader.